In [1]:
!pip install tensorflow-datasets

In [2]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\affinity\anaconda3\lib\site-packages (19.3.1)


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.models import load_model
from datetime import datetime
from keras import regularizers
#tf.enable_eager_execution()

Using TensorFlow backend.


In [4]:
train_data = tfds.load(name='cifar10:3.*.*', split='train[:80%]', as_supervised=True)
valid_data = tfds.load(name='cifar10:3.*.*', split='train[80%:]', as_supervised=True)
test_data = tfds.load(name='cifar10:3.*.*', split='test', as_supervised=True)

In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    image = tf.image.random_contrast(image, lower=2.0, upper=2.0)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_hue(image, max_delta = 0.8)
    image = tf.image.random_saturation(image, lower=2.0, upper=2.0)
    return image, label

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(128, 3, kernel_regularizer=regularizers.l2(0.001), activation='relu', padding='same', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),  # Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 하는 아이디어 중의 하나이다
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(256, 3, kernel_regularizer=regularizers.l2(0.001),activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3,kernel_regularizer=regularizers.l2(0.001), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(256, 3,kernel_regularizer=regularizers.l2(0.001), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(256, 3, kernel_regularizer=regularizers.l2(0.001),activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Conv2D(512, 3,kernel_regularizer=regularizers.l2(0.001), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(512, 3, kernel_regularizer=regularizers.l2(0.001),activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='softmax')
    
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 128)       512       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       590080    
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 256)         0

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

train_data = train_data.map(scale).shuffle(50000).repeat(4).batch(64) 
valid_data = valid_data.map(scale).batch(64)
test_data = test_data.map(scale).batch(64)

In [9]:
dt = datetime.now().strftime('%Y%m%d-%H%M%S')

cp_callback = tf.keras.callbacks.ModelCheckpoint(   
    'checkpoints/{}.hdf5'.format(dt),
    monitor='val_accuracy',
    save_best_only=True)

In [10]:
class MyCallback(tf.keras.callbacks.Callback):

    def on_epoch_start(self, epoch, logs=None):
        print('Epoch {} begins at {}: {}'.format(epoch, datetime.now().time(), logs))

    def on_epoch_end(self, epoch, logs=None):
        print('Epoch {} ends at {}: {}'.format(epoch, datetime.now().time(), logs))

In [11]:
history = model.fit(train_data,
                    epochs=25,
                    validation_data=valid_data,
                    verbose=0,
                    callbacks=[cp_callback, MyCallback()])

Epoch 0 ends at 16:15:12.093907: {'loss': 3.0550675556182862, 'accuracy': 0.4583625, 'val_loss': 18.845792284437046, 'val_accuracy': 0.3203}
Epoch 1 ends at 16:19:46.106354: {'loss': 2.0074566781044005, 'accuracy': 0.5946875, 'val_loss': 1.7870584210013127, 'val_accuracy': 0.5873}
Epoch 2 ends at 16:24:29.631514: {'loss': 1.464357743024826, 'accuracy': 0.68518126, 'val_loss': 1.486431313168471, 'val_accuracy': 0.6571}
Epoch 3 ends at 16:29:02.530321: {'loss': 1.2397050529241562, 'accuracy': 0.73244375, 'val_loss': 1.2138441751717002, 'val_accuracy': 0.7329}
Epoch 4 ends at 16:33:30.565408: {'loss': 1.1078958471059799, 'accuracy': 0.76259375, 'val_loss': 1.2304117907384398, 'val_accuracy': 0.7221}
Epoch 5 ends at 16:38:06.008965: {'loss': 1.0271761696338653, 'accuracy': 0.78361875, 'val_loss': 1.0313146034623408, 'val_accuracy': 0.7794}
Epoch 6 ends at 16:42:53.275781: {'loss': 0.9805133419990539, 'accuracy': 0.7969813, 'val_loss': 0.9990640464861682, 'val_accuracy': 0.7907}
Epoch 7 end

In [12]:
model.evaluate(test_data)

157/157 [==============================].9231 - accuracy: 0.76 - 0s 66ms/step - loss: 0.9691 - accuracy: 0.81 - 0s 60ms/step - loss: 0.9679 - accuracy: 0.81 - 0s 57ms/step - loss: 0.9299 - accuracy: 0.82 - 0s 55ms/step - loss: 0.9359 - accuracy: 0.80 - 0s 53ms/step - loss: 0.9170 - accuracy: 0.80 - 0s 51ms/step - loss: 0.9215 - accuracy: 0.81 - 0s 50ms/step - loss: 0.9561 - accuracy: 0.80 - 0s 49ms/step - loss: 0.9474 - accuracy: 0.80 - 0s 48ms/step - loss: 0.9503 - accuracy: 0.81 - 1s 47ms/step - loss: 0.9418 - accuracy: 0.81 - 1s 46ms/step - loss: 0.9495 - accuracy: 0.80 - 1s 45ms/step - loss: 0.9684 - accuracy: 0.80 - 1s 45ms/step - loss: 0.9682 - accuracy: 0.80 - 1s 45ms/step - loss: 0.9704 - accuracy: 0.80 - 1s 45ms/step - loss: 0.9744 - accuracy: 0.80 - 1s 45ms/step - loss: 0.9945 - accuracy: 0.79 - 1s 45ms/step - loss: 0.9986 - accuracy: 0.79 - 1s 44ms/step - loss: 0.9978 - accuracy: 0.79 - 1s 44ms/step - loss: 1.0105 - accuracy: 0.79 - 1s 44ms/step - loss: 0.9999 - accuracy: 0.

[0.998979410548119, 0.7878]

In [13]:
m = tf.keras.models.load_model('checkpoints/{}.hdf5'.format(dt))